# Ch14: Higher-Order Functions 高階函式 - 詳解範例

本檔案包含 5 個完整的詳解範例，展示高階函式在實際問題中的應用。

---

## 範例 1: 資料處理管道 - 使用 map/filter 建立 ETL Pipeline

### 問題描述

有一批原始的學生成績資料，需要進行以下處理：
1. 過濾掉分數為 None 的無效資料
2. 將分數從百分制轉換為五等第制 (A/B/C/D/F)
3. 統計各等第的人數

原始資料格式：
```python
[
    {'name': '小明', 'score': 85},
    {'name': '小華', 'score': None},
    {'name': '小美', 'score': 92},
    ...
]
```

### 解題步驟

In [ ]:
# 步驟 1: 準備原始資料
students = [
    {'name': '小明', 'score': 85},
    {'name': '小華', 'score': None},
    {'name': '小美', 'score': 92},
    {'name': '小強', 'score': 78},
    {'name': '小芳', 'score': None},
    {'name': '小傑', 'score': 67},
    {'name': '小玲', 'score': 58},
    {'name': '小軒', 'score': 95},
    {'name': '小怡', 'score': 73},
    {'name': '小涵', 'score': 45}
]

print("原始資料:")
for s in students:
    print(f"  {s}")

In [ ]:
# 步驟 2: 定義轉換函式

def score_to_grade(score):
    """將分數轉換為等第
    
    Args:
        score: 分數 (0-100)
    
    Returns:
        等第 (A/B/C/D/F)
    """
    if score >= 90:
        return 'A'
    elif score >= 80:
        return 'B'
    elif score >= 70:
        return 'C'
    elif score >= 60:
        return 'D'
    else:
        return 'F'

# 測試轉換函式
print("轉換函式測試:")
test_scores = [95, 85, 75, 65, 55]
for score in test_scores:
    grade = score_to_grade(score)
    print(f"  {score} 分 → {grade}")

In [ ]:
# 步驟 3: 建立資料處理管道

# 3.1 過濾掉無效資料 (score 為 None)
valid_students = list(filter(lambda s: s['score'] is not None, students))

print("過濾後的有效資料:")
for s in valid_students:
    print(f"  {s}")

print(f"\n有效資料數: {len(valid_students)} (原始: {len(students)})")

In [ ]:
# 3.2 轉換分數為等第
students_with_grade = list(map(
    lambda s: {
        'name': s['name'],
        'score': s['score'],
        'grade': score_to_grade(s['score'])
    },
    valid_students
))

print("\n轉換後的資料 (含等第):")
for s in students_with_grade:
    print(f"  {s['name']}: {s['score']}分 ({s['grade']})")

In [ ]:
# 3.3 統計各等第人數
from functools import reduce

# 方法 1: 使用 reduce 統計
grade_counts = reduce(
    lambda acc, s: {**acc, s['grade']: acc.get(s['grade'], 0) + 1},
    students_with_grade,
    {}
)

print("\n各等第人數統計 (使用 reduce):")
for grade in ['A', 'B', 'C', 'D', 'F']:
    count = grade_counts.get(grade, 0)
    print(f"  {grade}: {count} 人")

# 方法 2: 使用傳統方法對照
grade_counts_traditional = {}
for s in students_with_grade:
    grade = s['grade']
    grade_counts_traditional[grade] = grade_counts_traditional.get(grade, 0) + 1

print("\n驗證 (傳統方法):")
print(f"  結果相同: {grade_counts == grade_counts_traditional}")

In [ ]:
# 步驟 4: 整合成完整的管道函式

def process_student_data(raw_data):
    """完整的資料處理管道
    
    Args:
        raw_data: 原始學生資料列表
    
    Returns:
        dict: 包含處理後資料和統計結果
    """
    # Pipeline: 過濾 → 轉換 → 統計
    
    # 1. 過濾無效資料
    valid_data = list(filter(lambda s: s['score'] is not None, raw_data))
    
    # 2. 轉換分數為等第
    processed_data = list(map(
        lambda s: {
            'name': s['name'],
            'score': s['score'],
            'grade': score_to_grade(s['score'])
        },
        valid_data
    ))
    
    # 3. 統計等第分布
    grade_distribution = reduce(
        lambda acc, s: {**acc, s['grade']: acc.get(s['grade'], 0) + 1},
        processed_data,
        {}
    )
    
    return {
        'data': processed_data,
        'distribution': grade_distribution,
        'total_valid': len(valid_data),
        'total_invalid': len(raw_data) - len(valid_data)
    }

# 使用管道函式
result = process_student_data(students)

print("\n完整處理結果:")
print(f"有效資料: {result['total_valid']} 筆")
print(f"無效資料: {result['total_invalid']} 筆")
print("\n等第分布:")
for grade in ['A', 'B', 'C', 'D', 'F']:
    count = result['distribution'].get(grade, 0)
    print(f"  {grade}: {count} 人")

### 關鍵知識點

1. **管道式處理**: 使用 filter → map → reduce 形成清晰的資料處理流程
2. **Lambda 表達式**: 用於簡單的過濾和轉換邏輯
3. **函式組合**: 將多個小函式組合成完整的處理邏輯
4. **不可變性**: 每一步都產生新的資料結構，不修改原始資料

---

## 範例 2: 複雜排序 - 多層級排序條件 with lambda

### 問題描述

有一個產品列表，需要按照以下優先順序排序：
1. 首先按庫存狀態排序：有庫存 (in_stock=True) 在前
2. 其次按評分高低排序：評分高的在前
3. 最後按價格低至高排序：價格低的在前

### 解題步驟

In [ ]:
# 步驟 1: 準備產品資料
products = [
    {'name': '商品A', 'price': 1200, 'rating': 4.5, 'in_stock': True},
    {'name': '商品B', 'price': 800, 'rating': 4.8, 'in_stock': False},
    {'name': '商品C', 'price': 1500, 'rating': 4.2, 'in_stock': True},
    {'name': '商品D', 'price': 900, 'rating': 4.8, 'in_stock': True},
    {'name': '商品E', 'price': 1200, 'rating': 4.5, 'in_stock': False},
    {'name': '商品F', 'price': 600, 'rating': 4.0, 'in_stock': True},
    {'name': '商品G', 'price': 1100, 'rating': 4.8, 'in_stock': True},
    {'name': '商品H', 'price': 750, 'rating': 4.3, 'in_stock': False},
]

print("原始產品列表:")
for p in products:
    stock_status = "有貨" if p['in_stock'] else "缺貨"
    print(f"  {p['name']}: ${p['price']}, 評分 {p['rating']}, {stock_status}")

In [ ]:
# 步驟 2: 單一條件排序示範

# 2.1 只按價格排序
by_price = sorted(products, key=lambda p: p['price'])
print("\n只按價格排序 (低到高):")
for p in by_price[:3]:  # 顯示前 3 個
    print(f"  {p['name']}: ${p['price']}")

# 2.2 只按評分排序 (降序)
by_rating = sorted(products, key=lambda p: p['rating'], reverse=True)
print("\n只按評分排序 (高到低):")
for p in by_rating[:3]:
    print(f"  {p['name']}: 評分 {p['rating']}")

# 2.3 只按庫存狀態排序
by_stock = sorted(products, key=lambda p: p['in_stock'], reverse=True)
print("\n只按庫存狀態排序 (有貨優先):")
for p in by_stock[:3]:
    stock_status = "有貨" if p['in_stock'] else "缺貨"
    print(f"  {p['name']}: {stock_status}")

In [ ]:
# 步驟 3: 多層級排序 - 使用 tuple

# 排序鍵: (庫存狀態, 評分, 價格)
# - 庫存: False < True，但我們要 True 在前，所以用 not in_stock
# - 評分: 高的在前，所以用負數 -rating
# - 價格: 低的在前，直接用 price

sorted_products = sorted(
    products,
    key=lambda p: (not p['in_stock'], -p['rating'], p['price'])
)

print("\n多層級排序結果:")
print("(優先級: 有貨 > 高評分 > 低價格)\n")

for i, p in enumerate(sorted_products, 1):
    stock_status = "有貨" if p['in_stock'] else "缺貨"
    print(f"{i}. {p['name']}: ${p['price']}, 評分 {p['rating']}, {stock_status}")

In [ ]:
# 步驟 4: 建立可重用的排序函式

def sort_products(products, priority='default'):
    """根據不同策略排序產品
    
    Args:
        products: 產品列表
        priority: 排序策略
            - 'default': 有貨 > 高評分 > 低價格
            - 'price': 只按價格 (低到高)
            - 'rating': 只按評分 (高到低)
            - 'deal': 好評低價 (高評分 > 低價格)
    
    Returns:
        排序後的產品列表
    """
    sort_keys = {
        'default': lambda p: (not p['in_stock'], -p['rating'], p['price']),
        'price': lambda p: p['price'],
        'rating': lambda p: -p['rating'],
        'deal': lambda p: (-p['rating'], p['price'])
    }
    
    key_func = sort_keys.get(priority, sort_keys['default'])
    return sorted(products, key=key_func)

# 測試不同策略
print("\n策略: 好評低價 (不考慮庫存):")
deal_products = sort_products(products, 'deal')
for p in deal_products[:3]:
    print(f"  {p['name']}: ${p['price']}, 評分 {p['rating']}")

print("\n策略: 只看價格:")
cheap_products = sort_products(products, 'price')
for p in cheap_products[:3]:
    print(f"  {p['name']}: ${p['price']}")

In [ ]:
# 步驟 5: 使用 operator 模組優化效能
from operator import itemgetter, attrgetter

# itemgetter 比 lambda 效能更好
def sort_products_optimized(products):
    """使用 itemgetter 的優化版本"""
    return sorted(
        products,
        key=lambda p: (not p['in_stock'], -p['rating'], p['price'])
    )

# 效能比較
import time

# 建立大量測試資料
large_products = products * 1000

# 測試 lambda 版本
start = time.time()
sorted_products_lambda = sort_products(large_products, 'default')
time_lambda = time.time() - start

print(f"\n效能測試 ({len(large_products)} 個產品):")
print(f"Lambda 版本: {time_lambda:.4f} 秒")
print("\n提示: 對於簡單的鍵提取，itemgetter 通常更快")
print("      但對於複雜的轉換邏輯，lambda 更靈活")

### 關鍵知識點

1. **Tuple 排序**: Python 會依序比較 tuple 中的每個元素
2. **布林值排序**: False < True，用 `not` 可以反轉順序
3. **負數技巧**: 用 `-value` 實現降序排序
4. **排序穩定性**: Python 的排序是穩定的，相同鍵值的元素保持原有順序
5. **operator 模組**: itemgetter/attrgetter 可提升效能

---

## 範例 3: 裝飾器工廠 - 建立帶參數的裝飾器

### 問題描述

建立一個可以接受參數的裝飾器，用於限制函式的執行次數。
例如：`@limit_calls(3)` 表示該函式最多只能被呼叫 3 次。

### 解題步驟

In [ ]:
# 步驟 1: 理解裝飾器的三層結構

# 簡單裝飾器 (不帶參數)
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"呼叫 {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@simple_decorator
def greet(name):
    return f"你好, {name}!"

print("簡單裝飾器示範:")
result = greet("小明")
print(result)

In [ ]:
# 步驟 2: 建立帶參數的裝飾器 (裝飾器工廠)

def limit_calls(max_calls):
    """限制函式呼叫次數的裝飾器工廠
    
    Args:
        max_calls: 最大呼叫次數
    
    Returns:
        裝飾器函式
    """
    def decorator(func):
        # 使用列表儲存呼叫次數 (避免 nonlocal)
        call_count = [0]
        
        def wrapper(*args, **kwargs):
            call_count[0] += 1
            
            if call_count[0] > max_calls:
                raise RuntimeError(
                    f"{func.__name__} 已達到最大呼叫次數 {max_calls}"
                )
            
            print(f"呼叫 {func.__name__} (第 {call_count[0]}/{max_calls} 次)")
            return func(*args, **kwargs)
        
        # 提供重置功能
        wrapper.reset = lambda: call_count.__setitem__(0, 0)
        wrapper.get_count = lambda: call_count[0]
        
        return wrapper
    return decorator

# 測試裝飾器
@limit_calls(3)
def send_email(recipient):
    """發送郵件 (模擬)"""
    return f"郵件已發送給 {recipient}"

print("\n測試次數限制:")
try:
    print(send_email("user1@example.com"))
    print(send_email("user2@example.com"))
    print(send_email("user3@example.com"))
    print(send_email("user4@example.com"))  # 這次應該失敗
except RuntimeError as e:
    print(f"錯誤: {e}")

In [ ]:
# 步驟 3: 增強功能 - 添加時間重置
import time
from datetime import datetime, timedelta

def rate_limit(max_calls, time_window):
    """在指定時間窗口內限制呼叫次數
    
    Args:
        max_calls: 最大呼叫次數
        time_window: 時間窗口 (秒)
    """
    def decorator(func):
        call_history = []
        
        def wrapper(*args, **kwargs):
            now = datetime.now()
            
            # 移除超過時間窗口的記錄
            cutoff_time = now - timedelta(seconds=time_window)
            call_history[:] = [t for t in call_history if t > cutoff_time]
            
            # 檢查是否超過限制
            if len(call_history) >= max_calls:
                oldest_call = min(call_history)
                wait_time = (oldest_call + timedelta(seconds=time_window) - now).total_seconds()
                raise RuntimeError(
                    f"{func.__name__} 超過速率限制。"
                    f"請等待 {wait_time:.1f} 秒後再試。"
                )
            
            # 記錄這次呼叫
            call_history.append(now)
            
            print(f"呼叫 {func.__name__} ({len(call_history)}/{max_calls} 次 in {time_window}s)")
            return func(*args, **kwargs)
        
        return wrapper
    return decorator

# 測試速率限制
@rate_limit(max_calls=3, time_window=2)
def api_call(endpoint):
    """模擬 API 呼叫"""
    return f"API {endpoint} 回應成功"

print("\n測試速率限制 (2 秒內最多 3 次):")
try:
    print(api_call("/users"))
    print(api_call("/posts"))
    print(api_call("/comments"))
    print(api_call("/likes"))  # 這次應該失敗
except RuntimeError as e:
    print(f"錯誤: {e}")

In [ ]:
# 步驟 4: 建立通用的裝飾器工廠

def create_decorator(before=None, after=None, error_handler=None):
    """通用裝飾器工廠
    
    Args:
        before: 在函式執行前呼叫的函式
        after: 在函式執行後呼叫的函式
        error_handler: 錯誤處理函式
    """
    def decorator(func):
        def wrapper(*args, **kwargs):
            # 執行前處理
            if before:
                before(func, args, kwargs)
            
            try:
                # 執行函式
                result = func(*args, **kwargs)
                
                # 執行後處理
                if after:
                    after(func, result)
                
                return result
            
            except Exception as e:
                # 錯誤處理
                if error_handler:
                    return error_handler(func, e)
                raise
        
        return wrapper
    return decorator

# 使用通用工廠建立自訂裝飾器
def log_before(func, args, kwargs):
    print(f"→ 開始執行 {func.__name__}")

def log_after(func, result):
    print(f"← 完成執行 {func.__name__}, 結果: {result}")

def handle_error(func, error):
    print(f"✗ {func.__name__} 發生錯誤: {error}")
    return None

# 建立自訂裝飾器
logged = create_decorator(before=log_before, after=log_after, error_handler=handle_error)

@logged
def divide(a, b):
    return a / b

print("\n測試通用裝飾器:")
print("正常執行:")
result = divide(10, 2)

print("\n錯誤處理:")
result = divide(10, 0)

### 關鍵知識點

1. **裝飾器工廠**: 三層結構 (工廠 → 裝飾器 → wrapper)
2. **閉包應用**: 利用閉包儲存裝飾器的狀態
3. **可變容器**: 使用列表避免 nonlocal 關鍵字
4. **擴展功能**: 為 wrapper 添加額外方法 (reset, get_count)
5. **通用設計**: 建立可重用的裝飾器工廠模式

---

## 範例 4: 函式組合 - 文字處理管道

### 問題描述

建立一個文字處理系統，可以靈活組合不同的處理步驟：
- 移除空白
- 轉換大小寫
- 移除標點符號
- 分詞
- 過濾停用詞

### 解題步驟

In [ ]:
# 步驟 1: 定義基本的處理函式
import string

def strip_text(text):
    """移除頭尾空白"""
    return text.strip()

def lowercase(text):
    """轉換為小寫"""
    return text.lower()

def remove_punctuation(text):
    """移除標點符號"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def tokenize(text):
    """分詞 (以空格分割)"""
    return text.split()

# 測試基本函式
sample_text = "  Hello, World! This is PYTHON.  "
print("原始文字:", repr(sample_text))
print("移除空白:", repr(strip_text(sample_text)))
print("轉小寫:", repr(lowercase(strip_text(sample_text))))
print("移除標點:", repr(remove_punctuation(lowercase(strip_text(sample_text)))))

In [ ]:
# 步驟 2: 建立函式組合工具
from functools import reduce

def compose(*functions):
    """從右到左組合函式
    
    compose(f, g, h)(x) = f(g(h(x)))
    """
    def composed(x):
        return reduce(lambda acc, f: f(acc), reversed(functions), x)
    return composed

def pipe(*functions):
    """從左到右組合函式 (管道式)
    
    pipe(f, g, h)(x) = h(g(f(x)))
    """
    def piped(x):
        return reduce(lambda acc, f: f(acc), functions, x)
    return piped

# 建立處理管道
process_text = pipe(
    strip_text,
    lowercase,
    remove_punctuation
)

# 測試管道
result = process_text(sample_text)
print("\n使用管道處理:")
print(f"輸入: {repr(sample_text)}")
print(f"輸出: {repr(result)}")

In [ ]:
# 步驟 3: 建立停用詞過濾器

def create_stopwords_filter(stopwords):
    """建立停用詞過濾函式
    
    Args:
        stopwords: 停用詞集合
    
    Returns:
        過濾函式
    """
    stopwords_set = set(stopwords)
    
    def filter_stopwords(words):
        """過濾停用詞"""
        return [word for word in words if word not in stopwords_set]
    
    return filter_stopwords

# 定義英文停用詞
english_stopwords = {'a', 'an', 'the', 'is', 'are', 'was', 'were', 'this', 'that'}
filter_english_stopwords = create_stopwords_filter(english_stopwords)

# 測試停用詞過濾
words = ['this', 'is', 'a', 'python', 'tutorial']
filtered = filter_english_stopwords(words)
print("\n停用詞過濾:")
print(f"原始: {words}")
print(f"過濾後: {filtered}")

In [ ]:
# 步驟 4: 建立完整的文字處理系統

class TextProcessor:
    """文字處理器 - 支援靈活的管道組合"""
    
    def __init__(self):
        self.pipeline = []
    
    def add_step(self, func, name=None):
        """添加處理步驟"""
        step_name = name or func.__name__
        self.pipeline.append((step_name, func))
        return self  # 支援鏈式呼叫
    
    def process(self, text, verbose=False):
        """執行處理管道
        
        Args:
            text: 輸入文字
            verbose: 是否顯示每一步的結果
        """
        result = text
        
        if verbose:
            print(f"初始: {repr(result)}")
        
        for step_name, func in self.pipeline:
            result = func(result)
            if verbose:
                print(f"{step_name}: {repr(result)}")
        
        return result
    
    def clear(self):
        """清空管道"""
        self.pipeline.clear()
        return self

# 使用文字處理器
processor = TextProcessor()
processor.add_step(strip_text, "移除空白") \
         .add_step(lowercase, "轉小寫") \
         .add_step(remove_punctuation, "移除標點") \
         .add_step(tokenize, "分詞") \
         .add_step(filter_english_stopwords, "過濾停用詞")

# 處理文字
test_text = "  This is a PYTHON tutorial! Are you ready?  "
print("\n完整處理流程 (verbose=True):")
result = processor.process(test_text, verbose=True)
print(f"\n最終結果: {result}")

In [ ]:
# 步驟 5: 建立預設管道

def create_standard_processor(stopwords=None):
    """建立標準文字處理器"""
    processor = TextProcessor()
    processor.add_step(strip_text) \
             .add_step(lowercase) \
             .add_step(remove_punctuation) \
             .add_step(tokenize)
    
    if stopwords:
        filter_func = create_stopwords_filter(stopwords)
        processor.add_step(filter_func, "過濾停用詞")
    
    return processor

# 快速使用
quick_processor = create_standard_processor(stopwords=english_stopwords)

documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one."
]

print("\n批量處理文件:")
processed_docs = [quick_processor.process(doc) for doc in documents]
for i, (original, processed) in enumerate(zip(documents, processed_docs), 1):
    print(f"文件 {i}:")
    print(f"  原始: {original}")
    print(f"  處理後: {processed}")

### 關鍵知識點

1. **函式組合**: 將多個小函式組合成複雜的處理邏輯
2. **管道模式**: pipe 讓處理流程更直觀 (從左到右)
3. **高階函式工廠**: create_stopwords_filter 根據參數生成客製化函式
4. **鏈式呼叫**: 回傳 self 支援方法鏈
5. **可重用性**: 處理步驟可以靈活組合和重用

---

## 範例 5: 快取系統 - 建立快取裝飾器

### 問題描述

建立一個支援多種快取策略的裝飾器系統：
- 簡單快取 (無限制)
- LRU (Least Recently Used) 快取
- 帶過期時間的快取

### 解題步驟

In [ ]:
# 步驟 1: 簡單的記憶化裝飾器

def simple_cache(func):
    """簡單快取裝飾器"""
    cache = {}
    
    def wrapper(*args):
        if args not in cache:
            print(f"計算 {func.__name__}{args}")
            cache[args] = func(*args)
        else:
            print(f"快取命中 {func.__name__}{args}")
        return cache[args]
    
    wrapper.cache = cache
    wrapper.cache_info = lambda: {
        'size': len(cache),
        'items': list(cache.keys())
    }
    
    return wrapper

# 測試
@simple_cache
def fibonacci(n):
    """費氏數列"""
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print("簡單快取測試:")
print(f"fibonacci(6) = {fibonacci(6)}")
print(f"\n快取資訊: {fibonacci.cache_info()}")

In [ ]:
# 步驟 2: LRU 快取 (最近最少使用)
from collections import OrderedDict

def lru_cache(maxsize=128):
    """LRU 快取裝飾器
    
    Args:
        maxsize: 快取最大容量
    """
    def decorator(func):
        cache = OrderedDict()
        hits = [0]
        misses = [0]
        
        def wrapper(*args):
            # 檢查快取
            if args in cache:
                hits[0] += 1
                # 移動到最後 (標記為最近使用)
                cache.move_to_end(args)
                print(f"快取命中: {func.__name__}{args}")
                return cache[args]
            
            # 計算結果
            misses[0] += 1
            result = func(*args)
            print(f"計算: {func.__name__}{args}")
            
            # 加入快取
            cache[args] = result
            
            # 檢查容量限制
            if len(cache) > maxsize:
                # 移除最久未使用的項目 (第一個)
                oldest = next(iter(cache))
                print(f"移除最舊快取: {oldest}")
                cache.pop(oldest)
            
            return result
        
        def cache_info():
            return {
                'hits': hits[0],
                'misses': misses[0],
                'maxsize': maxsize,
                'current_size': len(cache),
                'hit_rate': hits[0] / (hits[0] + misses[0]) if (hits[0] + misses[0]) > 0 else 0
            }
        
        wrapper.cache_info = cache_info
        wrapper.cache_clear = lambda: cache.clear()
        
        return wrapper
    return decorator

# 測試 LRU 快取
@lru_cache(maxsize=3)
def expensive_computation(x):
    """耗時的計算 (模擬)"""
    import time
    time.sleep(0.1)
    return x * x

print("\nLRU 快取測試 (maxsize=3):")
print(expensive_computation(1))  # 計算
print(expensive_computation(2))  # 計算
print(expensive_computation(3))  # 計算
print(expensive_computation(1))  # 命中
print(expensive_computation(4))  # 計算，移除 2
print(expensive_computation(2))  # 計算 (已被移除)

print(f"\n快取統計: {expensive_computation.cache_info()}")

In [ ]:
# 步驟 3: 帶過期時間的快取
import time
from datetime import datetime, timedelta

def ttl_cache(ttl_seconds=60):
    """帶過期時間的快取
    
    Args:
        ttl_seconds: 快取存活時間 (秒)
    """
    def decorator(func):
        cache = {}
        timestamps = {}
        
        def wrapper(*args):
            now = time.time()
            
            # 檢查快取是否存在且未過期
            if args in cache:
                cached_time = timestamps[args]
                age = now - cached_time
                
                if age < ttl_seconds:
                    remaining = ttl_seconds - age
                    print(f"快取命中: {func.__name__}{args} (剩餘 {remaining:.1f}s)")
                    return cache[args]
                else:
                    print(f"快取過期: {func.__name__}{args} (已過 {age:.1f}s)")
                    # 清除過期快取
                    del cache[args]
                    del timestamps[args]
            
            # 計算並快取結果
            print(f"計算: {func.__name__}{args}")
            result = func(*args)
            cache[args] = result
            timestamps[args] = now
            
            return result
        
        wrapper.cache_info = lambda: {
            'size': len(cache),
            'ttl': ttl_seconds,
            'items': [
                {
                    'args': args,
                    'age': time.time() - timestamps[args],
                    'remaining': ttl_seconds - (time.time() - timestamps[args])
                }
                for args in cache
            ]
        }
        
        return wrapper
    return decorator

# 測試帶過期時間的快取
@ttl_cache(ttl_seconds=2)
def get_current_time():
    """獲取當前時間"""
    return datetime.now().strftime('%H:%M:%S')

print("\nTTL 快取測試 (ttl=2s):")
print(f"第 1 次: {get_current_time()}")  # 計算
time.sleep(1)
print(f"第 2 次: {get_current_time()}")  # 命中
time.sleep(1.5)
print(f"第 3 次: {get_current_time()}")  # 過期，重新計算

In [ ]:
# 步驟 4: 統一的快取介面

def cache(strategy='simple', **kwargs):
    """統一的快取裝飾器介面
    
    Args:
        strategy: 快取策略 ('simple', 'lru', 'ttl')
        **kwargs: 策略特定參數
    """
    strategies = {
        'simple': lambda: simple_cache,
        'lru': lambda: lru_cache(maxsize=kwargs.get('maxsize', 128)),
        'ttl': lambda: ttl_cache(ttl_seconds=kwargs.get('ttl', 60))
    }
    
    decorator_factory = strategies.get(strategy)
    if not decorator_factory:
        raise ValueError(f"未知的快取策略: {strategy}")
    
    return decorator_factory()

# 使用統一介面
@cache(strategy='lru', maxsize=5)
def factorial(n):
    """計算階乘"""
    if n <= 1:
        return 1
    return n * factorial(n-1)

print("\n使用統一快取介面:")
print(f"factorial(5) = {factorial(5)}")
print(f"\n快取統計: {factorial.cache_info()}")

### 關鍵知識點

1. **記憶化**: 快取函式結果以避免重複計算
2. **LRU 算法**: 使用 OrderedDict 實現最近最少使用策略
3. **TTL 機制**: 基於時間的快取過期
4. **快取統計**: 追蹤命中率和效能指標
5. **策略模式**: 統一介面支援多種快取策略

---

## 總結

本檔案展示了高階函式在實際應用中的 5 個完整範例：

1. **資料處理管道**: map/filter/reduce 的組合應用
2. **複雜排序**: 多層級排序條件和優化技巧
3. **裝飾器工廠**: 建立帶參數的裝飾器
4. **函式組合**: 靈活的文字處理管道
5. **快取系統**: 多種快取策略的實現

這些範例涵蓋了高階函式的核心概念和實用技巧，是理解和掌握函數式程式設計的重要基礎。

---